In [78]:
import pandas as pd
from sqlalchemy import create_engine, inspect
import numpy as np
import datetime
from config import *

In [79]:
df = pd.read_csv("NHL_Teams.csv")
df.drop(["Arena","Capacity","Joined","General manager","Head coach","Captain"],axis = 1, inplace=True)
df.head()

,Team,City,Conference,Division,Founded
0,Boston Bruins,"Boston, Massachusetts",Eastern,Atlantic,1924
1,Buffalo Sabres,"Buffalo, New York",Eastern,Atlantic,1970
2,Detroit Red Wings,"Detroit, Michigan",Eastern,Atlantic,1926
3,Florida Panthers,"Sunrise, Florida",Eastern,Atlantic,1993
4,Montreal Canadiens,"Montreal, Quebec",Eastern,Atlantic,1909


In [80]:
lk_conf=pd.DataFrame(df.Conference.unique())
lk_conf.columns=["conference_name"]
lk_conf.head()

,conference_name
0,Eastern
1,Western


In [82]:
connection_string = f"postgresql+psycopg2://{SQL_USERNAME}:{SQL_PASSWORD}@{SQL_IP}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

In [83]:
conn = engine.connect()
lk_conf.to_sql("lk_conference", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()

In [84]:
lk_div=df.groupby(["Conference","Division"]).size().reset_index()
lk_div=lk_div.drop(0,axis=1)
lk_div.columns=["conference_name","division_name"]
lk_div.head()

,conference_name,division_name
0,Eastern,Atlantic
1,Eastern,Metropolitan
2,Western,Central
3,Western,Pacific


In [85]:
query="""
select
*
from 
lk_conference

"""

In [86]:
conn = engine.connect()
sql_conf = pd.read_sql(query, con=conn)
conn.close()

sql_conf.head()

,id,conference_name,last_updated
0,1,Eastern,2020-12-22 01:28:45.939076
1,2,Western,2020-12-22 01:28:45.939076


In [87]:
lk_div_full=lk_div.merge(sql_conf,on="conference_name")
lk_div_full=lk_div_full[["id","division_name"]]
lk_div_full.rename(columns={"id":"conference_id"}, inplace=True )
lk_div_full.head()

,conference_id,division_name
0,1,Atlantic
1,1,Metropolitan
2,2,Central
3,2,Pacific


In [88]:
conn = engine.connect()
lk_div_full.to_sql("lk_division", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()

In [89]:
query="""
select
*
from 
lk_division

"""

In [90]:
conn = engine.connect()
sql_div = pd.read_sql(query, con=conn)
conn.close()

sql_div.head()

,id,conference_id,division_name,last_updated
0,1,1,Atlantic,2020-12-22 01:28:48.284605
1,2,1,Metropolitan,2020-12-22 01:28:48.284605
2,3,2,Central,2020-12-22 01:28:48.284605
3,4,2,Pacific,2020-12-22 01:28:48.284605


In [91]:
df.head()

,Team,City,Conference,Division,Founded
0,Boston Bruins,"Boston, Massachusetts",Eastern,Atlantic,1924
1,Buffalo Sabres,"Buffalo, New York",Eastern,Atlantic,1970
2,Detroit Red Wings,"Detroit, Michigan",Eastern,Atlantic,1926
3,Florida Panthers,"Sunrise, Florida",Eastern,Atlantic,1993
4,Montreal Canadiens,"Montreal, Quebec",Eastern,Atlantic,1909


In [92]:
df_teams = pd.read_csv("team_info.csv")
df_teams["Team"]=df_teams["shortName"].str.cat(df_teams.teamName, sep = " ")
df_teams.head()

,team_id,franchiseId,shortName,teamName,abbreviation,link,Team
0,24,32,Anaheim,Ducks,ANA,/api/v1/teams/24,Anaheim Ducks
1,53,28,Arizona,Coyotes,ARI,/api/v1/teams/53,Arizona Coyotes
2,11,35,Atlanta,Thrashers,ATL,/api/v1/teams/11,Atlanta Thrashers
3,6,6,Boston,Bruins,BOS,/api/v1/teams/6,Boston Bruins
4,7,19,Buffalo,Sabres,BUF,/api/v1/teams/7,Buffalo Sabres


In [93]:
df_full = df.merge(df_teams, on = "Team")
df_full = df_full[["Team", "City", "Conference", "Division", "Founded", "abbreviation"]]
df_full.columns = ["team_name", "city","conference_name","division_name","year_founded","team_abbr"]
df_full.head()

,team_name,city,conference_name,division_name,year_founded,team_abbr
0,Boston Bruins,"Boston, Massachusetts",Eastern,Atlantic,1924,BOS
1,Buffalo Sabres,"Buffalo, New York",Eastern,Atlantic,1970,BUF
2,Detroit Red Wings,"Detroit, Michigan",Eastern,Atlantic,1926,DET
3,Florida Panthers,"Sunrise, Florida",Eastern,Atlantic,1993,FLA
4,Montreal Canadiens,"Montreal, Quebec",Eastern,Atlantic,1909,MTL


In [94]:
team_full=df_full.merge(sql_conf,on="conference_name")
team_full=team_full[["id","team_name", "city","division_name","year_founded","team_abbr"]]
team_full.rename(columns={"id":"conference_id"}, inplace=True )
team_full.head()

,conference_id,team_name,city,division_name,year_founded,team_abbr
0,1,Boston Bruins,"Boston, Massachusetts",Atlantic,1924,BOS
1,1,Buffalo Sabres,"Buffalo, New York",Atlantic,1970,BUF
2,1,Detroit Red Wings,"Detroit, Michigan",Atlantic,1926,DET
3,1,Florida Panthers,"Sunrise, Florida",Atlantic,1993,FLA
4,1,Montreal Canadiens,"Montreal, Quebec",Atlantic,1909,MTL


In [95]:
team_full=team_full.merge(sql_div,on="division_name")
team_full=team_full[["id","team_name", "city","conference_id_x","year_founded","team_abbr"]]
team_full.rename(columns={"id":"division_id","conference_id_x":"conference_id"}, inplace=True )
team_full.head()

,division_id,team_name,city,conference_id,year_founded,team_abbr
0,1,Boston Bruins,"Boston, Massachusetts",1,1924,BOS
1,1,Buffalo Sabres,"Buffalo, New York",1,1970,BUF
2,1,Detroit Red Wings,"Detroit, Michigan",1,1926,DET
3,1,Florida Panthers,"Sunrise, Florida",1,1993,FLA
4,1,Montreal Canadiens,"Montreal, Quebec",1,1909,MTL


In [96]:
conn = engine.connect()
team_full.to_sql("teams", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()

In [117]:
player_df = pd.read_csv("players.csv")
player_df.groupby("team").nunique()

,Unnamed: 0,first_name,last_name,team,position
team,,,,,
ANA,22,19,22,1,4
ANA/CBJ,2,2,2,1,2
ANA/NSH,2,2,2,1,1
ARI,23,21,23,1,4
BOS,26,24,26,1,4
...,...,...,...,...,...
WPG,25,25,25,1,4
WPG/VGS,1,1,1,1,1
WSH,20,20,20,1,4


In [118]:
player_df = player_df.rename(columns={"team":"team_abbr"})
player_df['team_abbr'] = player_df['team_abbr'].str[:3]
player_df.groupby("team_abbr").nunique()

,Unnamed: 0,first_name,last_name,team_abbr,position
team_abbr,,,,,
ANA,26,23,26,1,4
ARI,23,21,23,1,4
BOS,30,27,29,1,4
BUF,27,25,27,1,4
CAR,27,26,27,1,4
CBJ,27,26,27,1,4
CGY,25,23,25,1,4
CHI,26,24,26,1,4
COL,26,25,26,1,4


In [119]:
player_df['team_abbr'] = player_df['team_abbr'].replace(['LA/'],'LA')
player_df['team_abbr'] = player_df['team_abbr'].replace(['NJ/'],'NJ')
player_df['team_abbr'] = player_df['team_abbr'].replace(['SJ/'],'SJ')

In [120]:
player_df.groupby("team_abbr").nunique()

,Unnamed: 0,first_name,last_name,team_abbr,position
team_abbr,,,,,
ANA,26,23,26,1,4
ARI,23,21,23,1,4
BOS,30,27,29,1,4
BUF,27,25,27,1,4
CAR,27,26,27,1,4
CBJ,27,26,27,1,4
CGY,25,23,25,1,4
CHI,26,24,26,1,4
COL,26,25,26,1,4


In [121]:
query=  """
        select
        *
        from 
        teams
        
        """

In [122]:
conn = engine.connect()
sql_teams = pd.read_sql(query, con=conn)
conn.close()

sql_teams.head()

,id,team_abbr,team_name,year_founded,city,conference_id,division_id,last_updated
0,1,BOS,Boston Bruins,1924,"Boston, Massachusetts",1,1,2020-12-22 01:28:53.504910
1,2,BUF,Buffalo Sabres,1970,"Buffalo, New York",1,1,2020-12-22 01:28:53.504910
2,3,DET,Detroit Red Wings,1926,"Detroit, Michigan",1,1,2020-12-22 01:28:53.504910
3,4,FLA,Florida Panthers,1993,"Sunrise, Florida",1,1,2020-12-22 01:28:53.504910
4,5,MTL,Montreal Canadiens,1909,"Montreal, Quebec",1,1,2020-12-22 01:28:53.504910


In [123]:
player_full = player_df.merge(sql_teams,on="team_abbr")
player_full

,Unnamed: 0,first_name,last_name,team_abbr,position,id,team_name,year_founded,city,conference_id,division_id,last_updated
0,0,Leon,Draisaitl,EDM,C,27,Edmonton Oilers,1972,"Edmonton, Alberta",2,4,2020-12-22 01:28:53.504910
1,1,Connor,McDavid,EDM,C,27,Edmonton Oilers,1972,"Edmonton, Alberta",2,4,2020-12-22 01:28:53.504910
2,35,Ryan,Nugent-Hopkins,EDM,C,27,Edmonton Oilers,1972,"Edmonton, Alberta",2,4,2020-12-22 01:28:53.504910
3,156,Tyler,Ennis,EDM,LW,27,Edmonton Oilers,1972,"Edmonton, Alberta",2,4,2020-12-22 01:28:53.504910
4,181,Zack,Kassian,EDM,RW,27,Edmonton Oilers,1972,"Edmonton, Alberta",2,4,2020-12-22 01:28:53.504910
...,...,...,...,...,...,...,...,...,...,...,...,...
793,693,Matt,Irwin,ANA,D,24,Anaheim Ducks,1993,"Anaheim, California",2,4,2020-12-22 01:28:53.504910
794,702,Sam,Carrick,ANA,C,24,Anaheim Ducks,1993,"Anaheim, California",2,4,2020-12-22 01:28:53.504910
795,710,Daniel,Sprong,ANA,RW,24,Anaheim Ducks,1993,"Anaheim, California",2,4,2020-12-22 01:28:53.504910
796,749,Jani,Hakanpaa,ANA,D,24,Anaheim Ducks,1993,"Anaheim, California",2,4,2020-12-22 01:28:53.504910


In [125]:
player_full=player_full[["first_name", "last_name","id","position"]]
player_full

,first_name,last_name,id,position
0,Leon,Draisaitl,27,C
1,Connor,McDavid,27,C
2,Ryan,Nugent-Hopkins,27,C
3,Tyler,Ennis,27,LW
4,Zack,Kassian,27,RW
...,...,...,...,...
793,Matt,Irwin,24,D
794,Sam,Carrick,24,C
795,Daniel,Sprong,24,RW
796,Jani,Hakanpaa,24,D


In [126]:
player_full = player_full.rename(columns={"id":"team_id"})
player_full

,first_name,last_name,team_id,position
0,Leon,Draisaitl,27,C
1,Connor,McDavid,27,C
2,Ryan,Nugent-Hopkins,27,C
3,Tyler,Ennis,27,LW
4,Zack,Kassian,27,RW
...,...,...,...,...
793,Matt,Irwin,24,D
794,Sam,Carrick,24,C
795,Daniel,Sprong,24,RW
796,Jani,Hakanpaa,24,D


In [127]:
conn = engine.connect()
player_full.to_sql("player", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()

In [128]:
stats_df = pd.read_csv("player_stats.csv")
stats_df

,Unnamed: 0,first_name,last_name,GP,G,A,PTS,PIM
0,0,Leon,Draisaitl,71,43,67,110,18
1,1,Connor,McDavid,64,34,63,97,28
2,2,David,Pastrnak,70,48,47,95,40
3,3,Artemi,Panarin,69,32,63,95,20
4,4,Nathan,MacKinnon,69,35,58,93,12
...,...,...,...,...,...,...,...,...
793,793,Paul,LaDue,2,0,1,1,0
794,794,Pontus,Aberg,5,0,1,1,0
795,795,Ryan,MacInnis,10,0,1,1,0
796,796,Clark,Bishop,5,0,1,1,4


In [130]:
query=  """
        select
        *
        from 
        player
        
        """

In [131]:
conn = engine.connect()
sql_player = pd.read_sql(query, con=conn)
conn.close()

sql_player.head()

,id,first_name,last_name,team_id,position,last_updated
0,1,Leon,Draisaitl,27,C,2020-12-22 01:41:27.397651
1,2,Connor,McDavid,27,C,2020-12-22 01:41:27.397651
2,3,Ryan,Nugent-Hopkins,27,C,2020-12-22 01:41:27.397651
3,4,Tyler,Ennis,27,LW,2020-12-22 01:41:27.397651
4,5,Zack,Kassian,27,RW,2020-12-22 01:41:27.397651


In [135]:
stats_full = pd.merge(stats_df, sql_player,  how='left', left_on=['last_name','first_name'], right_on = ['last_name','first_name'])
stats_full.head()

,Unnamed: 0,first_name,last_name,GP,G,A,PTS,PIM,id,team_id,position,last_updated
0,0,Leon,Draisaitl,71,43,67,110,18,1,27,C,2020-12-22 01:41:27.397651
1,1,Connor,McDavid,64,34,63,97,28,2,27,C,2020-12-22 01:41:27.397651
2,2,David,Pastrnak,70,48,47,95,40,28,1,RW,2020-12-22 01:41:27.397651
3,3,Artemi,Panarin,69,32,63,95,20,58,13,LW,2020-12-22 01:41:27.397651
4,4,Nathan,MacKinnon,69,35,58,93,12,81,18,C,2020-12-22 01:41:27.397651


In [136]:
stats_full=stats_full[["id","GP", "G","A","PTS","PIM"]]
stats_full

,id,GP,G,A,PTS,PIM
0,1,71,43,67,110,18
1,2,64,34,63,97,28
2,28,70,48,47,95,40
3,58,69,32,63,95,20
4,81,69,35,58,93,12
...,...,...,...,...,...,...
793,437,2,0,1,1,0
794,186,5,0,1,1,0
795,723,10,0,1,1,0
796,361,5,0,1,1,4


In [137]:
stats_full = stats_full.rename(columns={"id":"player_id","GP":"games_played","G":"goals","A":"assists","PTS":"points","PIM":"penalty_minutes"})
stats_full

,player_id,games_played,goals,assists,points,penalty_minutes
0,1,71,43,67,110,18
1,2,64,34,63,97,28
2,28,70,48,47,95,40
3,58,69,32,63,95,20
4,81,69,35,58,93,12
...,...,...,...,...,...,...
793,437,2,0,1,1,0
794,186,5,0,1,1,0
795,723,10,0,1,1,0
796,361,5,0,1,1,4


In [138]:
conn = engine.connect()
stats_full.to_sql("player_stats", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()